# **Traveler Insights**

A medida que el turismo se vuelve cada vez más digital y las reseñas en línea influyen fuertemente en las decisiones de viaje, las plataformas digitales juegan un papel crucial en la comprensión de las preferencias y comportamientos de los turistas. A través de Traveler Insights, tu misión es desarrollar un modelo capaz de clasificar el sentimiento de estas reseñas en tres categorías principales: positivo, neutral o negativo. Esta tarea es clave para ayudar a la industria turística a comprender mejor las emociones de los visitantes, permitiéndoles adaptar sus ofertas para mejorar las experiencias y optimizar sus servicios.

Participantes:

Daniel Isaías Cruz González

Jaider Castilla Babilonia

# Librerías Python

---

In [37]:
# Importando librerias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# ML


# Contenido de datos

*
*
*
*

# Lectura y Análisis de los datos

---

In [10]:
# Carga del dataset "TRAIN"
data_train = pd.read_csv('dataset/train.csv')
data_train.head(5)

,ID,Sitio,Índice del lugar,Nombre del lugar,Enlace del lugar,Municipio,Valoración,Valoraciones,Precio,Comentario,Fecha,Votos a favor,Votos en contra,Valoración_num,Sentimiento
0,380,Tripadvisor,50,Isla Múcura,https://www.tripadvisor.co/Attraction_Review-g...,San Onofre,4.5/5,281,Precio Desconocido,La isla es muy hermosa y tiene unas aguas muy ...,24 de agosto de 2017,0,0,4.5,neutral
1,196,Foursquare,30,Doki's comidas rápidas (Majagual),https://es.foursquare.com/v/dokis-comidas-r%C3...,Sincelejo,7.4/10,13 valoraciones,$,"Lahamburguesa de polloy elpatacónDoki, son del...","Abril 12, 2015",0,0,7.4,positivo
2,723,Tripadvisor,56,Playa la Coquerita,https://www.tripadvisor.co/Attraction_Review-g...,Coveñas,4.0/5,28,Precio Desconocido,"La playa es preciosa, de aguas mansas y playas...",14 de febrero de 2020,0,0,4.0,neutral
3,522,Tripadvisor,50,Isla Múcura,https://www.tripadvisor.co/Attraction_Review-g...,San Onofre,4.5/5,281,Precio Desconocido,es un paraíso de playa Para llegar a la isla l...,3 de octubre de 2012,0,0,4.5,neutral
4,1126,Tripadvisor,74,Covenas Diving Co.,https://www.tripadvisor.co/Attraction_Review-g...,Coveñas,5.0/5,259,Precio Desconocido,Es muy buena escuela de buceo; excelente los l...,4 de marzo de 2018,0,0,5.0,neutral


In [11]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818 entries, 0 to 817
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                818 non-null    int64  
 1   Sitio             818 non-null    object 
 2   Índice del lugar  818 non-null    int64  
 3   Nombre del lugar  818 non-null    object 
 4   Enlace del lugar  817 non-null    object 
 5   Municipio         818 non-null    object 
 6   Valoración        818 non-null    object 
 7   Valoraciones      818 non-null    object 
 8   Precio            818 non-null    object 
 9   Comentario        818 non-null    object 
 10  Fecha             818 non-null    object 
 11  Votos a favor     818 non-null    int64  
 12  Votos en contra   818 non-null    int64  
 13  Valoración_num    818 non-null    float64
 14  Sentimiento       818 non-null    object 
dtypes: float64(1), int64(4), object(10)
memory usage: 96.0+ KB


In [12]:
# Carga del dataset "TEST"
data_test = pd.read_csv('dataset/test.csv')
data_test.head(5)

,ID,Sitio,Índice del lugar,Nombre del lugar,Enlace del lugar,Municipio,Valoración,Valoraciones,Precio,Comentario,Fecha,Votos a favor,Votos en contra,Valoración_num,Sentimiento
0,56,Foursquare,2,Playas de Coveñas,https://es.foursquare.com/v/playas-de-cove%C3%...,Coveñas,8.8/10,241 valoraciones,Precio Desconocido,Un lugar para vacacionar? Coveñas es relax total!,"Noviembre 3, 2012",4,0,8.8,positivo
1,1083,Tripadvisor,74,Covenas Diving Co.,https://www.tripadvisor.co/Attraction_Review-g...,Coveñas,5.0/5,259,Precio Desconocido,De regalo del Día del Padre me regalaron esta ...,5 de julio de 2019,2,0,5.0,neutral
2,262,Foursquare,43,Nonnina Pizza,https://foursquare.com/v/nonnina-pizza/4d17ec0...,Sincelejo,6.8/10,12 valoraciones,$,"Recomiendo las pastas con tocineta, son realme...","Enero 15, 2014",0,0,6.8,neutral
3,447,Tripadvisor,50,Isla Múcura,https://www.tripadvisor.co/Attraction_Review-g...,San Onofre,4.5/5,281,Precio Desconocido,"Increíble! Precioso lugar para descansar, muy ...",20 de febrero de 2015,0,0,4.5,neutral
4,59,Foursquare,2,Playas de Coveñas,https://es.foursquare.com/v/playas-de-cove%C3%...,Coveñas,8.8/10,241 valoraciones,Precio Desconocido,Muy bueno deli .,"Noviembre 21, 2013",1,0,8.8,positivo


In [13]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818 entries, 0 to 817
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                818 non-null    int64  
 1   Sitio             818 non-null    object 
 2   Índice del lugar  818 non-null    int64  
 3   Nombre del lugar  818 non-null    object 
 4   Enlace del lugar  817 non-null    object 
 5   Municipio         818 non-null    object 
 6   Valoración        818 non-null    object 
 7   Valoraciones      818 non-null    object 
 8   Precio            818 non-null    object 
 9   Comentario        818 non-null    object 
 10  Fecha             818 non-null    object 
 11  Votos a favor     818 non-null    int64  
 12  Votos en contra   818 non-null    int64  
 13  Valoración_num    818 non-null    float64
 14  Sentimiento       818 non-null    object 
dtypes: float64(1), int64(4), object(10)
memory usage: 96.0+ KB


#### ANÁLISIS DE NULOS

In [15]:
#total_nulos_train = 
print(f"número total de registros nulos en \"train\" : {data_train.isnull().sum().sum()}")
#total_nulos_test = 
print(f"número total de registros nulos en \"test\" : {data_test.isnull().sum().sum()}")
#nulos_por_columna = df.isnull().sum()
#nulos_por_columna

número total de registros nulos en "train" : 1
número total de registros nulos en "test" : 0


In [16]:
nulos_por_columna = data_train.isnull().sum()
nulos_por_columna

ID                  0
Sitio               0
Índice del lugar    0
Nombre del lugar    0
Enlace del lugar    1
Municipio           0
Valoración          0
Valoraciones        0
Precio              0
Comentario          0
Fecha               0
Votos a favor       0
Votos en contra     0
Valoración_num      0
Sentimiento         0
dtype: int64

In [17]:
nulos_por_columna = data_test.isnull().sum()
nulos_por_columna

ID                  0
Sitio               0
Índice del lugar    0
Nombre del lugar    0
Enlace del lugar    0
Municipio           0
Valoración          0
Valoraciones        0
Precio              0
Comentario          0
Fecha               0
Votos a favor       0
Votos en contra     0
Valoración_num      0
Sentimiento         0
dtype: int64

**Conclusión:** El único dataset que posee valores nulos es el de entrenamiento. Sin embargo, la característica dónde se encuentre dicho valor nulo ("Enlace del lugar") es prescindible para el análisis. Por lo que se procederá a eliminar dicha característica de ambos dataset.


In [19]:
data_train = data_train.drop("Enlace del lugar", axis=1)
data_test = data_test.drop("Enlace del lugar", axis=1)

In [20]:
data_train.head()

,ID,Sitio,Índice del lugar,Nombre del lugar,Municipio,Valoración,Valoraciones,Precio,Comentario,Fecha,Votos a favor,Votos en contra,Valoración_num,Sentimiento
0,380,Tripadvisor,50,Isla Múcura,San Onofre,4.5/5,281,Precio Desconocido,La isla es muy hermosa y tiene unas aguas muy ...,24 de agosto de 2017,0,0,4.5,neutral
1,196,Foursquare,30,Doki's comidas rápidas (Majagual),Sincelejo,7.4/10,13 valoraciones,$,"Lahamburguesa de polloy elpatacónDoki, son del...","Abril 12, 2015",0,0,7.4,positivo
2,723,Tripadvisor,56,Playa la Coquerita,Coveñas,4.0/5,28,Precio Desconocido,"La playa es preciosa, de aguas mansas y playas...",14 de febrero de 2020,0,0,4.0,neutral
3,522,Tripadvisor,50,Isla Múcura,San Onofre,4.5/5,281,Precio Desconocido,es un paraíso de playa Para llegar a la isla l...,3 de octubre de 2012,0,0,4.5,neutral
4,1126,Tripadvisor,74,Covenas Diving Co.,Coveñas,5.0/5,259,Precio Desconocido,Es muy buena escuela de buceo; excelente los l...,4 de marzo de 2018,0,0,5.0,neutral


In [21]:
data_test.head()

,ID,Sitio,Índice del lugar,Nombre del lugar,Municipio,Valoración,Valoraciones,Precio,Comentario,Fecha,Votos a favor,Votos en contra,Valoración_num,Sentimiento
0,56,Foursquare,2,Playas de Coveñas,Coveñas,8.8/10,241 valoraciones,Precio Desconocido,Un lugar para vacacionar? Coveñas es relax total!,"Noviembre 3, 2012",4,0,8.8,positivo
1,1083,Tripadvisor,74,Covenas Diving Co.,Coveñas,5.0/5,259,Precio Desconocido,De regalo del Día del Padre me regalaron esta ...,5 de julio de 2019,2,0,5.0,neutral
2,262,Foursquare,43,Nonnina Pizza,Sincelejo,6.8/10,12 valoraciones,$,"Recomiendo las pastas con tocineta, son realme...","Enero 15, 2014",0,0,6.8,neutral
3,447,Tripadvisor,50,Isla Múcura,San Onofre,4.5/5,281,Precio Desconocido,"Increíble! Precioso lugar para descansar, muy ...",20 de febrero de 2015,0,0,4.5,neutral
4,59,Foursquare,2,Playas de Coveñas,Coveñas,8.8/10,241 valoraciones,Precio Desconocido,Muy bueno deli .,"Noviembre 21, 2013",1,0,8.8,positivo


#### ANÁLISIS DE VALORES ÚNICOS

In [23]:
# Analizar las columnas y sus valores
for col in data_train.columns:
    unique_vals = data_train[col].nunique()
    print(f"Columna '{col}' tiene {unique_vals} valores únicos.")

Columna 'ID' tiene 818 valores únicos.
Columna 'Sitio' tiene 2 valores únicos.
Columna 'Índice del lugar' tiene 69 valores únicos.
Columna 'Nombre del lugar' tiene 73 valores únicos.
Columna 'Municipio' tiene 9 valores únicos.
Columna 'Valoración' tiene 23 valores únicos.
Columna 'Valoraciones' tiene 43 valores únicos.
Columna 'Precio' tiene 4 valores únicos.
Columna 'Comentario' tiene 809 valores únicos.
Columna 'Fecha' tiene 688 valores únicos.
Columna 'Votos a favor' tiene 16 valores únicos.
Columna 'Votos en contra' tiene 2 valores únicos.
Columna 'Valoración_num' tiene 23 valores únicos.
Columna 'Sentimiento' tiene 3 valores únicos.


**Conclusiones:** Tomando como punto de partida lo que se nos solicita en el reto, decidimos que las siguientes carácterísticas no son relevantes en el análisis de sentimientos a las reseñas:

* **ID** 
* **Sitio**
* **Indice del lugar**
* **Nombre del lugar**
* **Municipio**
* **Valoraciones**
* **Precio**
* **Fecha**
* **Valoración_num**
* ****

#### EDA VARIABLES CATEGÓRICAS

In [29]:
categorical_cols = data_train.select_dtypes(include=['object']).columns.tolist()

print("Columnas de lass variables categóricas:")
for col in categorical_cols:
    print(" -",col)

Columnas de lass variables categóricas:
 - Sitio
 - Nombre del lugar
 - Municipio
 - Valoración
 - Valoraciones
 - Precio
 - Comentario
 - Fecha
 - Sentimiento


In [41]:
def graficos_eda_categoricos(cat):
    
    #Calculamos el número de filas que necesitamos
    from math import ceil
    filas = ceil(cat.shape[1] / 2)

    #Definimos el gráfico
    f, ax = plt.subplots(nrows = filas, ncols = 2, figsize = (16, filas * 6))

    #Aplanamos para iterar por el gráfico como si fuera de 1 dimensión en lugar de 2
    ax = ax.flat 

    #Creamos el bucle que va añadiendo gráficos
    for cada, variable in enumerate(cat):
        try:
            cat[variable].value_counts().plot.barh(ax = ax[cada])
            ax[cada].set_title(variable, fontsize = 12, fontweight = "bold")
            ax[cada].tick_params(labelsize = 12)
        except:
            pass

In [43]:
graficos_eda_categoricos(data_train.select_dtypes('O'))

Error in callback <function _draw_all_if_interactive at 0x000002A20E4ECB80> (for post_execute), with arguments args (),kwargs {}:


ValueError: 
$$
^
ParseException: Expected end of text, found '$'  (at char 0), (line:1, col:1)

ValueError: 
$$
^
ParseException: Expected end of text, found '$'  (at char 0), (line:1, col:1)

<Figure size 1600x3000 with 10 Axes>

In [26]:
# Eliminar columnas no relevantes para el análisis
data_clean = data_train.drop(columns=["ID", 'Sitio', 'Nombre del lugar', "Índice del lugar", "Nombre del lugar",'Municipio', 'Precio','Fecha', "Valoración_num"])
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818 entries, 0 to 817
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Valoración       818 non-null    object
 1   Valoraciones     818 non-null    object
 2   Comentario       818 non-null    object
 3   Votos a favor    818 non-null    int64 
 4   Votos en contra  818 non-null    int64 
 5   Sentimiento      818 non-null    object
dtypes: int64(2), object(4)
memory usage: 38.5+ KB
